## 생성 모델 개념

기존 합성곱 신경망에서 다룬 이미지 분류, 이미지 검출 등은 입력 이미지(x)가 있을 때 그에 따른 정답(y)를 찾는 것이었습니다. 예를 들어 개와 고양이 이미지 데이터셋이 주어졌을 때, 그 이미지를 개와 고양이로 분류하는 문제들을 다루었습니다. 이렇게 `이미지를 분류하는 것을 판별 모델(discriminative model)` 이라고 합니다. 일반적으로 판별자 모델에서는 이미지를 정확히 분류(구별) 하고자 해당 이미지를 대표하는 특성들을 잘 찾는 것을 목표로 합니다. 예를 들어 개와 고양이를 구별하려면 개의 귀, 꼬리 등 특성을 찾는 것이 중요합니다.

그리고 판별자 모델에서 추출한 특성들의 조합을 이용하여 새로운 개와 고양이 이미지를 생성 할 수 있는데 이것을 `생성 모델(generative model)`이라고 합니다. 즉, 생성 모델은 입력 이미지에 대한 데이터 분포 p(x)를 학습하여 새로운 이미지(새로운 이미지이면서 기존 이미지에서 특성을 추출했기 때문에 최대한 입력 이미지와 유사한 이미지)를 생성하는 것을 목표로 합니다.

![](../Static/682.jpg)

## 생성 모델의 유형
생성 모델의 유형에는 다음 그림과 같이 모델의 확률 변수를 구하는 `번형 오토인코더 모델`과 확률 변수를 이용하지 않는 `GAN 모델`이 있습니다.  

![](../Static/683.jpg)  

다시 정리하면 생성 모델은 크게 명시적 방법(explicit density)와 암시적 방법(implicit density)으로 분류할 수 있습니다. 명시적 방법은 확률 변수 p(x)를 정의하여 사용합니다. 대표적인 모델로 `변형 오토인코더(variational autoencoder)`가 있습니다. 암시적 방법은 확률 변수 p(x)에 대한 정의없이 p(x)를 샘플링 하여 사용합니다. 대표적인 모델로 `GAN(Generative Adversarial Network)`이 있습니다.

변형 오토인코더는 이미지의 잠재 공간(latent space)에서 샘플링하여 완전히 새로운 이미지나 기존 이미지를 변형하는 방식으로 학습을 진행합니다. GAN은 생성자와 판별자가 서로 경쟁하면서 가짜 이미지를 진짜 이미지와 최대한 비슷하게 만들도록 학습을 진행합니다.

변형 오토인코더부터 살펴보겠습니다.

## 오토인코더란

오토인코더는 단순히 입력을 출력으로 복사하는 신경망으로 은닉층의 노드 수가 입력 값보다 적은 것이 특징입니다. 따라서 입력과 출력이 동일한 이미지라고 예상할 수 있습니다. 하지만 왜 입력을 출력으로 복사하는 방법을 사용할까요? 바로 은닉층 때문입니다. 오토인코더의 은닉층은 입력과 출력의 뉴런보다 훨씬 적습니다. 즉, 적은 수의 은닉층 뉴런으로 데이터를 가장 잘 표현할 수 있는 방법이 오토인코더입니다. 오토인코더는 네 가지 주요 부분으로 구성됩니다.

* 인코더 : 인지 네트워크(recognition network)라고도 하며, 특성에 대한 학습을 수행하는 부분입니다.

* 은닉층(병목층) : 모델의 뉴런 개수가 최소인 계층입니다. 이 계층에서는 차원이 가장 낮은 입력 데이터의 압축 표현이 포함됩니다.

* 디코더 : 생성 네트워크(generative network)라고도 하며, 이 부분은 병목층에서 압축된 데이터를 원래대로 재구성(reconstruction)하는 역할을 합니다. 즉, 최대한 입력에 가까운 출력을 생성하도록 합니다.

* 손실재구성 : 오토인코더는 다음 그림과 같이 입력층과 출력층의 뉴런 개수가 동일하다는 것만 제외하면 일반적으로 다층 퍼셉트로(MLP)과 구조가 동일합니다. 오토인코더는 압축된 입력을 출력층에서 재구성하며, 손실 함수는 입력과 출력(인코더와 디코더)의 차이를 가지고 계산합니다.

    ![](../Static/684.jpg)

오토인코더가 중요한 이유는 다음 세 가지 때문입니다.

* 데이터 압축 : 데이터 압축은 메모리 측면에서 상당한 장점입니다. 오토인코더를 이용하여 이미지나 음성 파일의 중요 특성만 압축하면 용량도 작고 품질도 더 좋아집니다.

* 차원의 저주(curse of dimensionality) 예방 : 차원의 저주 문제를 예방할 수 있습니다. 오토인코더는 특성 개수를 줄여 주기 때문에 데이터 차원이 감소하여 차원의 저주를 피할 수 있습니다.

* 특성 추출: 오토인코더는 `비지도 학습`으로 자동으로 중요한 특성을 찾아 줍니다. 예를 들어 눈모양, 털 색, 꼬리 길이 등 개의 중요한 특성을 자동으로 찾아 줍니다.

이제 파이토치에서 오토인코더를 구현해 볼 텐데, 그동안 진행했던 모델과는 다르게 조금 복잡할 수 있으니 반드시 예제를 실습하면서 이해하기 바랍니다.

예제는 다음 그림과 같은 구조의 오토인코더를 사용합니다.

![](../Static/686.jpg)